This function computes the interpolant for the instantaneous vorticity barrier equation:

\begin{equation}
 \mathbf{x}'=\nu \rho \mathbf{J} \nabla \dfrac{D}{Dt}\hat{\omega} (\mathbf{x}, t), \label{eq: vorticity} \tag{1}
\end{equation}

where $ \nu $ is the viscosity, $ \rho $ the density of the fluid, $ \mathbf{J} = \begin{pmatrix} 0 && 1 \\ -1 && 0 \end{pmatrix} $ and $ \hat{\omega} $ is the scalar valued vorticity in 2D.
 
 For simplicity we can normalize equation $ \ref{eq: vorticity} $ by $ \nu $ and $ \rho $ and thus simply omit them as they play no role when exploring the structures of the field $ \mathbf{x'} $.
 
 We compute $ \mathbf{\nabla}\dfrac{D}{Dt}\hat{\omega}(\mathbf{x},t) $ by advecting the trajectory starting at $ \mathbf{x} $ over a very short segment in order to compute the acceleration of the particle at time 't'. From there we can then evalute the right hand side of equation \ref{eq: vorticity} according to:
 
 \begin{equation}
 \nabla \dfrac{D}{Dt}\hat{\omega}(\mathbf{x},t) = \nabla \dfrac{D}{Dt} (\nabla \times \mathbf{u}) = \nabla (\nabla \times \dfrac{D}{Dt} \mathbf{u}) = \nabla (\nabla \times \mathbf{a}) \tag{2},
 \end{equation} where $ \mathbf{a} $ is the acceleration and $ \mathbf{u} $ is the velocity.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t0 | float | time at which to evaluate instantaneous vorticity barrier equation|
| X | array (Ny, Nx)| X-meshgrid with Ny=Nx=1024 |
| Y | array (Ny, Nx)| Y-meshgrid with Ny=Nx=1024 |
| Interpolant_u | RectangularBivariateSpline object | Interpolant for x-component of velocity field |
| Interpolant_v | RectangularBivariateSpline object | Interpolant for y-component of velocity field |
| periodic | list (3,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br /> periodic[2]: periodicity in time <br />|
| defined_domain | array (Nx, Ny) | points on the meshgrid where velocity field is defined |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| dt_data | float | time spacing of the velocity data |
| aux_grid | list (2,) | aux_grid[0]: dx_auxiliary spacing <br /> aux_grid[1]: dy_auxiliary spacing |
| Interpolant_Jgrad_curl_acc | list (2,) | Interpolant[0]: Interpolant for x-component of $ \mathbf{x}' $ <br /> Interpolant[1]: Interpolant for y-component of $ \mathbf{x}' $ |

In [10]:
# import sys and os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

/Users/enalex/OneDrive/TBarrier/TBarrier


In [12]:
# Import package for parallel computing
from joblib import Parallel, delayed

# Import package for progress bar
from tqdm.notebook import tqdm

# import Rectangular bivariate spline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

# Import package for computing trajectories/velocity along trajectories
from ipynb.fs.defs.integration_dFdt import integration_dFdt

# Import numpy 
import numpy as np

# Import math tools
from math import pi

In [1]:
def InstantaneousActiveVorticity(t0, X, Y, Interpolant_u, Interpolant_v, periodic, defined_domain, bool_unsteady, time_data, aux_grid, Ncores, verbose=False):
    '''
    Compute interpolant of right-hand-side of Eulerian vorticity barrier equation.
    
    Parameters:
        t:              float,  time instant  
        x:              array (2,Npoints),  array of ICs
        X:              array (NY, NX)  X-meshgrid of data domain
        Y:              array (NY, NX)  Y-meshgrid of data domain
        Interpolant_u:  Interpolant object for u(x, t)
        Interpolant_v:  Interpolant object for v(x, t)
        periodic:       list of 3 bools, periodic[i] is True if the flow is periodic in the ith coordinate. Time is i=3.
        defined_domain: array(NY, NX), meshgrid defining domain where velocity field is defined. If 1 --> velocity field is defined elseif 0 --> velocity field is not defined.
        bool_unsteady:  bool, specifies if velocity field is unsteady/steady
        time_data:      array(1,NT), time data        
        aux_grid:       list(2,), specifies spacing of x/y auxiliary grid components
        Ncores:         int, Number of cores used for parallel computing
        verbose:        bool, if True, function reports progress at every 100th iteration
        
    Returns:
        Interpolant_Jgrad_curl_acc: list(2,) of Interpolant-objects for right-hand-side of Eulerian vorticity barrier equation.
        Interpolant_Jgrad_curl_acc[0] --> Interpolant for x-component
        Interpolant_Jgrad_curl_acc[1] --> Interpolant for y-component
    '''
    
    # auxiliary grid
    rho_x = aux_grid[0]
    rho_y = aux_grid[1]
    
    # compute D/DT \omega by advecting the particles over a short period of time starting at 't' and 
    # then calculate D/DT \omega along the (ideally infinitesimally) short segment of the trajectory from the acceleration of the particle
    
    # This value can be even smaller. It just takes more time but computation becomes more accurate
    dt = 0.1
    
    # Compute (ideally infinitesimally) short trajectories starting at $ t0 $
    tspan = np.arange(t0, t0+2.1*dt, dt)
    
    Fmap = np.zeros((2, 3, X.shape[0]*Y.shape[1]))
    
    x0 = np.array([X.ravel(), Y.ravel()]).reshape(2,-1)
    
    dFdt = integration_dFdt(tspan, x0, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data, verbose=False)[1]
    
    acc_x = ((dFdt[1,0,:]-dFdt[0,0,:])/dt).reshape(X.shape[0], X.shape[1])
    acc_y = ((dFdt[1,1,:]-dFdt[0,1,:])/dt).reshape(X.shape[0], X.shape[1])
    
    # compute interpolant for acc_x, acc_y over meshgrid X, Y    
    Interpolant_acc_x = RBS(Y[:,0], X[0,:], acc_x, kx = 1, ky = 1)
    Interpolant_acc_y = RBS(Y[:,0], X[0,:], acc_y, kx = 1, ky = 1)
    
    # Compute curl of instantaneous acceleration (acc_x, acc_y)
    curl_acc = acc_x.copy()*0
    for i in range(X.shape[0]):
        for j in range(Y.shape[0]):
            curl_acc[i, j] = (Interpolant_acc_y(Y[i, j], X[i, j]+rho_x)- Interpolant_acc_y(Y[i, j], X[i, j]-rho_x))/(2*rho_x)
            curl_acc[i, j] -= (Interpolant_acc_x(Y[i, j]+rho_y, X[i, j])- Interpolant_acc_x(Y[i, j]-rho_y, X[i, j]))/(2*rho_y)

    # Compute interpolants for curl_acc
    Interpolant_curl_acc = RBS(Y[:,0], X[0,:], curl_acc, kx = 1, ky = 1)
    
    Jgrad_curl_acc = np.zeros((X.shape[0], X.shape[1], 2))
    
    # compute x'=J grad(curl_acc), with J = [[0, 1], [-1, 0]]
    for i in range(X.shape[0]):
        
        if verbose and i%100==0:
            print("Percentage completed: ", np.around(np.around(i/X.shape[0], 4)*100, 2))

        for j in range(Y.shape[1]):
            
            # apply periodic boundary conditions to point 'x'
            x = np.array([X[i, j], Y[i, j]])
            
            # evaluate grad(omega) at 'x' using auxiliary grid
            xR = x[0] + rho_x
            curl_accR = Interpolant_curl_acc(x[1], xR)[0][0]
            
            xL = x[0] - rho_x
            curl_accL = Interpolant_curl_acc(x[1], xL)[0][0]
            
            xU = x[1] + rho_y
            curl_accU = Interpolant_curl_acc(xU, x[0])[0][0]
            
            xD = x[1] - rho_y
            curl_accD = Interpolant_curl_acc(xD, x[0])[0][0]
            
            Jgrad_curl_acc[i, j, 0] = -(curl_accU - curl_accD)/(2*rho_y)
            Jgrad_curl_acc[i, j, 1] = (curl_accR - curl_accL)/(2*rho_x)

    Interpolant_Jgrad_curl_acc = []
    Interpolant_Jgrad_curl_acc.append(RBS(Y[:,0], X[0,:], Jgrad_curl_acc[:,:,0]))
    Interpolant_Jgrad_curl_acc.append(RBS(Y[:,0], X[0,:], Jgrad_curl_acc[:,:,1]))
    
    return Interpolant_Jgrad_curl_acc